# Selenium爬取

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
import os
import re
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import json
import random
import time
import win32con
import win32api
import win32gui

selenium爬取mhtml

In [11]:
# 随机延迟1到3秒
delay = random.uniform(1, 3)

user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/12.0.3 Safari/605.1.15"
# Options参数指定
chrome_options = Options()
# chrome_options.add_argument('--headless')
chrome_options.add_argument('window-size=1920x1080')
# chrome_options.add_argument('disable-gpu')  # 在无头模式下禁用GPU加速
chrome_options.add_argument('--user-agent=%s' % user_agent)
chrome_options.add_argument('--save-page-as-mhtml')  # 启用保存为MHTML的选项

# 添加options对象，剩下和普通selenium基本一致了
driver = webdriver.Chrome(executable_path="C:\\Program Files\\Google\\Chrome\\Application\\chromedriver.exe",options=chrome_options)
start_url="http://www.people.com.cn/"
print(start_url)
driver.get(start_url)


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 117
Current browser version is 119.0.6045.124 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe


In [ ]:
'''
爬取html
'''
driver.refresh()
#res = driver.execute_cdp_cmd('Page.captureSnapshot', {})
time.sleep(delay)
win32api.keybd_event(17, 0, 0, 0)  # 按下ctrl
win32api.keybd_event(83, 0, 0, 0)  # 按下s
win32api.keybd_event(83, 0, win32con.KEYEVENTF_KEYUP, 0)  # 释放s
win32api.keybd_event(17, 0, win32con.KEYEVENTF_KEYUP, 0)  # 释放ctrl
# hd = win32gui.FindWindow(u"#32770", u"另存为")
# win32gui.SetForegroundWindow(hd)
time.sleep(delay)
win32api.keybd_event(13, 0, 0, 0)  # 按下enter
win32api.keybd_event(13, 0, win32con.KEYEVENTF_KEYUP, 0)  # 释放enter
time.sleep(delay)

In [ ]:
'''
爬取全部图片
'''
try:
    # 定位所有图片元素
    wait = WebDriverWait(driver, 10)  # 最大等待时间为10秒
    img_elements = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, 'img')))
except Exception as e:
            print("发生异常:", e)

output_folder="E:\\XMU\\大三上\\计网\\实验\\实验三\\爬虫图片库"
# 创建文件夹（如果不存在的话）
os.makedirs(output_folder, exist_ok=True)

# 遍历所有图片元素，下载并保存图片
for index, img_element in enumerate(img_elements):
    img_url = img_element.get_attribute('src')
    img_name = f'image_{index}.jpg'  # 可以根据需要修改图片保存的文件名规则
    img_path = os.path.join(output_folder, img_name)
    with open(img_path, 'wb') as img_file:
        img_content = requests.get(img_url).content
        img_file.write(img_content)


In [ ]:
driver.close()

# Socket爬取

In [13]:
import socket
import os
import re
from urllib.parse import urlparse

# 定义要连接的目标URL和端口号
target_host = "www.people.com.cn"
target_port = 80

# 建立TCP连接
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((target_host, target_port))

# 发送GET请求
request = f"GET / HTTP/1.1\r\nHost: {target_host}\r\n\r\n"
client_socket.send(request.encode())

# 接收服务器响应数据
response = b""
while True:
    data = client_socket.recv(4096)
    if not data:
        break
    response += data

# 关闭连接
client_socket.close()

# 将响应数据以字符串形式返回，忽略无法解码的部分
html_content=response.decode("utf-8", errors="ignore")

# 将网页内容保存到txt文件中
with open("people_com_cn.txt", "w", encoding="utf-8") as file:
    file.write(html_content)

print("网页内容已保存到people_com_cn.txt文件中。")



UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc8 in position 826: invalid continuation byte

Socket爬取问题，utf-8与字符不兼容

# BeautifulSoup爬取

In [ ]:
import sys
import numpy as np
import json
import re
import requests
import pandas as pd
import datetime
from bs4 import BeautifulSoup
import base64
import os
import random
import time

user_agents = ['Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:34.0) Gecko/20100101 Firefox/34.0',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/534.57.2 (KHTML, like Gecko) Version/5.1.7 Safari/534.57.2',
                   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
                   'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER',
                   'Mozilla/5.0 (Windows NT 5.1) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 SE 2.X MetaSr 1.0'
                   ]
cookie={'cookie':'bid=kRRUP5Adrsc; _ga=GA1.3.1583431493.1679359048; _gid=GA1.3.240421151.1679359048; _ga=GA1.1.1583431493.1679359048; page_style="mobile"; dbcl2="215291240:+lGgZ069L0g"; _pk_ses.100001.a7dd=*; ck=AT7V; _ga_RXNMP372GL=GS1.1.1679406549.4.1.1679408190.60.0.0; _pk_id.100001.a7dd=0f38c905a23f4f70.1679359049.4.1679408190.1679402067.; _gat=1'}

headers = { 
        'User-Agent': random.choice(user_agents)
    }
url='http://www.people.com.cn/' 
res = requests.get(url,headers=headers,cookies=cookie)
sp = BeautifulSoup(res.text,'lxml') 

In [ ]:
# 获取网页的HTML内容
html_content = sp.prettify()

# 将HTML内容保存到txt文件
with open("people_com_cn.html", "w", encoding="utf-8") as file:
    file.write(html_content)

print("网页内容已保存到people_com_cn.html文件中。")


网页内容已保存到people_com_cn.html文件中。


美丽汤爬取html的问题:html乱码